In [288]:
import numpy as np
import pandas as pd
import re
from scipy.stats import mode

In [289]:
train = pd.read_csv('~/datasets/titanic/train.csv')
test = pd.read_csv('~/datasets/titanic/test.csv')
y_train = train.pop('Survived')
train['source']='train'
test['source']='test'
df = pd.concat([train, test],ignore_index=True)
df.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [290]:
df.apply(lambda x: sum(x.isnull()))

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [291]:
df = df.drop(['PassengerId','Ticket'], axis=1)
df.tail(5)

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
1304,NaN,NaN,S,8.0500,"Spector, Mr. Woolf",0,3,male,0,NaN
1305,39.0,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0,1,female,0,NaN
1306,38.5,NaN,S,7.2500,"Saether, Mr. Simon Sivertsen",0,3,male,0,NaN
1307,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0,3,male,0,NaN
1308,NaN,NaN,C,22.3583,"Peter, Master. Michael J",1,3,male,1,NaN


In [292]:
df.Age.fillna(df.Age.mean(), inplace = True) 
df.Fare.fillna(df.Fare.mean(), inplace = True)
df.Cabin.fillna('U', inplace = True)
df.Embarked.fillna(mode(df.Embarked).mode[0], inplace = True)
df.Survived.fillna(mode(df.Survived).mode[0], inplace = True)
df.apply(lambda x: sum(x.isnull()))

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Age         0
Cabin       0
Embarked    0
Fare        0
Name        0
Parch       0
Pclass      0
Sex         0
SibSp       0
Survived    0
dtype: int64

In [293]:
df.tail(5)

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
1304,29.881138,U,S,8.0500,"Spector, Mr. Woolf",0,3,male,0,0.0
1305,39.000000,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0,1,female,0,0.0
1306,38.500000,U,S,7.2500,"Saether, Mr. Simon Sivertsen",0,3,male,0,0.0
1307,29.881138,U,S,8.0500,"Ware, Mr. Frederick",0,3,male,0,0.0
1308,29.881138,U,C,22.3583,"Peter, Master. Michael J",1,3,male,1,0.0


In [294]:
df['Deck'] = df['Cabin'].apply(lambda x: x[0])
df = df.drop('Cabin',axis=1)

In [295]:
df.Deck.value_counts()

U    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Deck, dtype: int64

In [296]:
def AClass(x):
    if x < 19.0:
        return 1
    elif x >= 19.0 and x < 25.0:
        return 2
    elif x >= 25.0 and x < 29.881:
        return 3
    elif x >= 29.881 and x < 30.5:
        return 4
    elif x >= 30.4 and x < 40.1:
        return 5
    else:
        return 6

df['Age_Category'] = df['Age'].apply(lambda x: AClass(x)) 
df = df.drop('Age', axis=1)

In [297]:
def FClass(x):
    if x < 7.775:
        return 1
    elif x >= 7.775 and x < 8.662:
        return 2
    elif x >= 8.662 and x < 14.454:
        return 3
    elif x >= 14.454 and x < 26.0:
        return 4
    elif x >= 26.0 and x < 53.1:
        return 5
    else:
        return 6

df['Fare_Category'] = df['Fare'].apply(lambda x: FClass(x)) 
df = df.drop('Fare', axis=1)

In [298]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [299]:
def IAClass(x):
    if x > 1:
        return 0
    else:
        return 1

df['IsAlone'] = df['FamilySize'].apply(lambda x: IAClass(x)) 
df = df.drop(['SibSp','Parch','FamilySize'], axis=1)

In [300]:
df['Title'] = df['Name'].str.extract(r',\s(.*)\.', expand=False)
Title_Dictionary = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"

                    }


df['Title'] = df['Title'].apply(lambda x: df.Title.map(Title_Dictionary))
df = df.drop('Name', axis=1)

In [301]:
df.Sex = df.Sex.map({'male':1,'female':0})

In [302]:
df.tail(5)

,Embarked,Pclass,Sex,Survived,Deck,Age_Category,Fare_Category,IsAlone,Title
1304,S,3,1,0.0,U,4,2,1,Mr
1305,C,1,0,0.0,C,5,6,1,Mr
1306,S,3,1,0.0,U,5,1,1,Mr
1307,S,3,1,0.0,U,4,2,1,Mr
1308,C,3,1,0.0,U,4,4,0,Mr


In [303]:
categorical_columns = [x for x in df.dtypes.index if df.dtypes[x]=='object']
categorical_columns

['Embarked', 'Deck', 'Title']

In [304]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Embarked','Title','Deck']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df = pd.get_dummies(df, columns=['Pclass','Embarked','Title','Deck','Age_Category','Fare_Category'])
df.head()

,Sex,Survived,IsAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2,Title_0,...,Age_Category_3,Age_Category_4,Age_Category_5,Age_Category_6,Fare_Category_1,Fare_Category_2,Fare_Category_3,Fare_Category_4,Fare_Category_5,Fare_Category_6
0,1,0.0,0,0,0,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,0,1.0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,1.0,1,0,0,1,0,0,1,1,...,1,0,0,0,0,1,0,0,0,0
3,0,1.0,0,1,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,1
4,1,0.0,1,0,0,1,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0


In [305]:
df.Survived = df.Survived.astype(int)
df.tail()

,Sex,Survived,IsAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2,Title_0,...,Age_Category_3,Age_Category_4,Age_Category_5,Age_Category_6,Fare_Category_1,Fare_Category_2,Fare_Category_3,Fare_Category_4,Fare_Category_5,Fare_Category_6
1304,1,0,1,0,0,1,0,0,1,1,...,0,1,0,0,0,1,0,0,0,0
1305,0,0,1,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1
1306,1,0,1,0,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1307,1,0,1,0,0,1,0,0,1,1,...,0,1,0,0,0,1,0,0,0,0
1308,1,0,0,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,1,0,0


In [307]:
train = df.loc[df['source']=="train"]
test = df.loc[df['source']=="test"]

train = train.drop('source',axis=1)
test = test.drop('source',axis=1)

X_train = train.loc[:,:]

X_test = test.loc[:,:]

In [308]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth = 5)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [309]:
y_pred = classifier.predict(X_test)

In [310]:
y_pred

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,

In [312]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7824074074074074

In [313]:
from sklearn.metrics import accuracy_score
print('Training accuracy: ', accuracy_score(y_train, classifier.predict(X_train)))
print('Validation accuracy: ', accuracy_score(y_test, classifier.predict(X_test)))

Training accuracy:  0.8255416191562144
Validation accuracy:  0.7824074074074074
